# Geographische Koordinaten deutscher Hochschulen

In diesem Beispiel werden einem Datensatz mit Adressen geographische Koordinaten hinzugefügt.

Datenquellen:

 - Liste der Hochschulen in Deutschland vom [Hochschulkompass](https://www.hochschulkompass.de/hochschulen/downloads.html)
 - Geodaten vom Kartendienst [Bing maps](https://www.bingmapsportal.com/)


Die Rechte an den Daten verbleiben bei den Urhebern!

In [1]:
# Liste der Hochschulen herunterladen

!wget https://www.hs-kompass.de/kompass/xml/download/hs_liste.txt

--2022-08-17 17:02:32--  https://www.hs-kompass.de/kompass/xml/download/hs_liste.txt
Resolving www.hs-kompass.de (www.hs-kompass.de)... 85.214.208.248
Connecting to www.hs-kompass.de (www.hs-kompass.de)|85.214.208.248|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 99170 (97K) [text/plain]
Saving to: 'hs_liste.txt'

     0K .......... .......... .......... .......... .......... 51%  881K 0s
    50K .......... .......... .......... .......... ......    100% 2,21M=0,08s

2022-08-17 17:02:32 (1,22 MB/s) - 'hs_liste.txt' saved [99170/99170]



In [2]:
import pandas as pd  # zur Arbeit mit Dataframes

# Textdatei in Dataframe einlesen

df = pd.read_csv(
  'hs_liste.txt',     # Dateiname
  sep = "\t",         # Spalten sind durch Tabulator getrennt
  encoding="latin_1", # Kodierung "Western Europe"
  dtype={"Postleitzahl (Hausanschrift)": str, "Postleitzahl (Postanschrift)": str} # PLZ als String, damit führende Nullen nicht verschwinden
  
)

In [3]:
# Whitespace und Umlaute aus Spaltennamen entfernen
# (optional)

df.rename(columns = {
    'Trägerschaft':'Traegerschaft',
    'Anzahl Studierende':'Anzahl_Studierende',
    'Gründungsjahr':'Gruendungsjahr',
    'Postleitzahl (Hausanschrift)':'PLZ',
    'Ort (Hausanschrift)':'Ort',
    'Straße':'Adresse',
    'Postleitzahl (Postanschrift)':'PLZ_Postanschrift',
    'Ort (Postanschrift)':'Ort_Postanschrift',
    'Home Page':'Web',
    'Mitglied HRK':'Mitglied_HRK'}, inplace = True)

Um Daten aus der Bing Maps Api abzurufen wird ein API-Key benötigt.
Eine Anleitung zum erstellen eines Keys gibt es [hier](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key
).

In [4]:
# Bing Maps Api Key hier einfügen!

apiKey = "meinApiKey";

In [5]:
from time import sleep
from urllib.parse import quote # zum enkodieren der Adressen
import requests # zum Abruf der API

def request_coords(plz, address):
    url = f"http://dev.virtualearth.net/REST/v1/Locations/DE/{plz}/{quote(address.strip(' .'))}?key={apiKey}"
    for retry in range(5):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                break
            if response.status_code == 404:
                print(f"404! ({plz}, {address})")
                sleep(2)
        except:
            print(f"Retry! ({plz}, {address})")
                
    try:
        coords = response.json()["resourceSets"][0]["resources"][0]["point"]["coordinates"]
    except:
        print(f"Couldn't get coords for {plz}, {address}. I added a [0,0] instead.")
        coords = [0, 0] 

    return coords


In [6]:
lat = []
long = []

for i, row in df.iterrows():
    coords = request_coords(row["PLZ"], row["Adresse"])
    lat.append(coords[0])
    long.append(coords[1])

df["lat"] = lat
df["long"] = long

In [7]:
df.head(3)

,Hs-Nr.,Hochschulkurzname,Hochschulname,Hochschultyp,Traegerschaft,Bundesland,Anzahl_Studierende,Gruendungsjahr,Promotionsrecht,Habilitationsrecht,...,Postfach,PLZ_Postanschrift,Ort_Postanschrift,Telefonvorwahl,Telefon,Fax,Web,Mitglied_HRK,lat,long
0,2,Aachen FH,Fachhochschule Aachen,Fachhochschulen / HAW,öffentlich-rechtlich,Nordrhein-Westfalen,14792.0,1971.0,Nein,Nein,...,100560,52005,Aachen,241,6009-0,6009-51090,http://www.fh-aachen.de,1,50.755605,6.095995
1,1,Aachen TH,Rheinisch-Westfälische Technische Hochschule A...,Universitäten,öffentlich-rechtlich,Nordrhein-Westfalen,47511.0,1870.0,Ja,Ja,...,NaN,52056,Aachen,241,80-1,80-92312,https://www.rwth-aachen.de,1,50.777638,6.077977
2,3,Aalen H,Hochschule Aalen - Technik und Wirtschaft,Fachhochschulen / HAW,öffentlich-rechtlich,Baden-Württemberg,5750.0,1962.0,Nein,Nein,...,1728,73428,Aalen,7361,576-0,576-2250,http://www.hs-aalen.de,1,48.838147,10.073161


In [8]:
df.to_csv("hochschulen.csv", index=False)